In [11]:
import pandas as pd

In [12]:
df = pd.read_json('/media/sidhu/Samsung 1TB/sysmon.json', lines=True, chunksize=100000)

- This is a hackish loop to convert my sysmon data that I exported from elastic.
- It removes elastic metafield data, adds some more structured data that was previously a string in another field
- It then saves all the data as csv files to be imported by the new elastic instance

In [14]:
for ind, chunk in enumerate(df):
    chunk.drop(['_index', '_type', '_id', '_score'], axis=1, inplace=True)
    json_df = pd.io.json.json_normalize(chunk['_source'].tolist())
    
    # Add message column information to the rest of the data
    dict_data = []
    for items in json_df['message'].tolist():
        temp_dict = {}
        for item in str(items).split('\n'):
            split = item.split(':')
            
            # Since the image contains the full path of the process
            # It gets split on ":" so we add a condition to handle it
            # Ignore the message "Network Connection Detected"
            if len(split) >= 2:
                if split[0] == 'Image' and len(split) > 2:
                    temp_dict[split[0]] = split[1] + ':' + split[2]
                elif split[0].endswith('detected'):
                    pass
                else:
                    temp_dict[split[0]] = split[1].strip()
        dict_data.append(temp_dict)

    message_data = pd.io.json.json_normalize(dict_data)
    full_data = pd.concat([json_df, message_data], axis=1).drop('message', axis=1)
    json_df.to_json(f'/home/sidhu/data_files/sysmon-{ind}.json', orient="records", lines=True)

    if ind % 10 == 0:
        print(ind)

0
10
20
30
40
50
60
70


In [6]:
test = pd.read_csv('/home/sidhu/data_files/sysmon-0.csv')

/home/sidhu/.virtualenvs/datascience/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (75,76,77,78,79,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,97,98,99,100,101,102,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,127,128,129,130,131,132,134,135,136,137,138,139,140,141,142,143,145,146,147,148,149,150,151,152,153,154,156,157,158,162,164) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
test

,Unnamed: 0,@timestamp,message,ecs.version,host.id,host.name,host.hostname,host.architecture,host.os.version,host.os.family,...,winlog.event_data.AddressLength,winlog.event_data.Address,winlog.event_data.QueryName,winlog.event_data.ProcessCreationTime,winlog.event_data.Type,winlog.event_data.ReturnCode,winlog.event_data.CountOfCredentialsReturned,winlog.event_data.ReadOperation,winlog.event_data.ClientProcessId,winlog.event_data.TargetName
0,0,2019-10-17T04:20:46.168Z,Network connection detected:\nRuleName: \nUtcT...,1.1.0,b66e662a-0e28-4126-aa9c-b2e11f057d0f,DESKTOP-CS3NIGE,DESKTOP-CS3NIGE,x86_64,10.0,windows,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2019-10-17T04:20:46.168Z,Network connection detected:\nRuleName: \nUtcT...,1.1.0,b66e662a-0e28-4126-aa9c-b2e11f057d0f,DESKTOP-CS3NIGE,DESKTOP-CS3NIGE,x86_64,10.0,windows,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2019-10-17T04:20:46.168Z,Network connection detected:\nRuleName: \nUtcT...,1.1.0,b66e662a-0e28-4126-aa9c-b2e11f057d0f,DESKTOP-CS3NIGE,DESKTOP-CS3NIGE,x86_64,10.0,windows,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2019-10-17T04:20:46.168Z,Network connection detected:\nRuleName: \nUtcT...,1.1.0,b66e662a-0e28-4126-aa9c-b2e11f057d0f,DESKTOP-CS3NIGE,DESKTOP-CS3NIGE,x86_64,10.0,windows,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2019-10-17T04:20:46.168Z,Network connection detected:\nRuleName: \nUtcT...,1.1.0,b66e662a-0e28-4126-aa9c-b2e11f057d0f,DESKTOP-CS3NIGE,DESKTOP-CS3NIGE,x86_64,10.0,windows,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,2019-10-17T18:59:37.601Z,Network connection detected:\nRuleName: \nUtcT...,1.1.0,b66e662a-0e28-4126-aa9c-b2e11f057d0f,DESKTOP-CS3NIGE,DESKTOP-CS3NIGE,x86_64,10.0,windows,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99996,99996,2019-10-17T18:59:37.602Z,Network connection detected:\nRuleName: \nUtcT...,1.1.0,b66e662a-0e28-4126-aa9c-b2e11f057d0f,DESKTOP-CS3NIGE,DESKTOP-CS3NIGE,x86_64,10.0,windows,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99997,99997,2019-10-17T18:59:37.602Z,Network connection detected:\nRuleName: \nUtcT...,1.1.0,b66e662a-0e28-4126-aa9c-b2e11f057d0f,DESKTOP-CS3NIGE,DESKTOP-CS3NIGE,x86_64,10.0,windows,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99998,99998,2019-10-17T18:59:37.602Z,Network connection detected:\nRuleName: \nUtcT...,1.1.0,b66e662a-0e28-4126-aa9c-b2e11f057d0f,DESKTOP-CS3NIGE,DESKTOP-CS3NIGE,x86_64,10.0,windows,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
display(test.columns)

Index(['Unnamed: 0', '@timestamp', 'message', 'ecs.version', 'host.id',
       'host.name', 'host.hostname', 'host.architecture', 'host.os.version',
       'host.os.family',
       ...
       'winlog.event_data.AddressLength', 'winlog.event_data.Address',
       'winlog.event_data.QueryName', 'winlog.event_data.ProcessCreationTime',
       'winlog.event_data.Type', 'winlog.event_data.ReturnCode',
       'winlog.event_data.CountOfCredentialsReturned',
       'winlog.event_data.ReadOperation', 'winlog.event_data.ClientProcessId',
       'winlog.event_data.TargetName'],
      dtype='object', length=165)